In [179]:
%matplotlib widget
import spb
import sympy as sp
from sympy import sin, cos, pi, sqrt, acos
sp.init_printing()

In [180]:
# Helper functions
from typing import List


def homogeneous(rotation: sp.Matrix = sp.eye(3), translation: sp.Matrix = sp.zeros(3, 1)) -> sp.Matrix:
    return rotation.row_join(translation).col_join(sp.Matrix([[0, 0, 0, 1]]))

def dh(rotation, twist, displacement, offset):
    rotation_mat = sp.Matrix([
        [cos(rotation), -sin(rotation)*cos(twist),  sin(rotation)*sin(twist)],
        [sin(rotation), cos(rotation)*cos(twist),   -cos(rotation)*sin(twist)],
        [0,             sin(twist),                 cos(twist)],
    ])
    translation = sp.Matrix([
        [offset*cos(rotation)],
        [offset*sin(rotation)],
        [displacement],
    ])
    return rotation_mat, translation

def rotation(homogeneous: sp.Matrix):
    return homogeneous[:3, :3]

def translation(homogeneous: sp.Matrix):
    return homogeneous[:3, 3:]

def jacobian(transforms: List[sp.Matrix], joint_types: List[sp.Matrix], base_z: sp.Matrix = sp.Matrix([0, 0, 1])):
    transforms_chained = [homogeneous()]
    z_unit_vecs = []
    # chain the transforms
    for i, transform in enumerate(transforms):
        transforms_chained.append(transforms_chained[-1] * transform)
    # apply rotation of chained transforms to base z
    for chained_transform in transforms_chained:
        z_unit_vecs.append(rotation(chained_transform) * base_z)
    
    assert len(transforms_chained) == len(z_unit_vecs)

    jacobian = sp.zeros(6, len(transforms))
    for i, (transform, joint_type) in enumerate(zip(transforms, joint_types)):
        p = 1 if joint_type == 'revolute' else 0
        # linear velocity
        # handle revolute
        if p == 1:
            jacobian[:3, i] = z_unit_vecs[i].cross(translation(transforms_chained[-1]) - translation(transforms_chained[i]))
        # handle prismatic
        else:
            jacobian[:3, i] = z_unit_vecs[i]

        # angular velocity
        jacobian[3:, i] = p * z_unit_vecs[i]

    return jacobian

In [181]:
turret_angle, shoulder_angle, wrist_angle = sp.symbols('q_1 q_2 q_3')
shoulder_height = sp.Symbol('d_{o,p}')
shoulder_offset = sp.Symbol('d_{p,q}')
arm_length = sp.Symbol('d_{q,r}')
forearm_offset = sp.Symbol('d_{r,s}')
forearm_length = sp.Symbol('d_{s,t}')

turret_rot, turret_trans = dh(turret_angle, -pi/2, shoulder_height, 0)
shoulder_rot, shoulder_trans = dh(shoulder_angle, 0, shoulder_offset, arm_length)
arm_rot, arm_trans = dh(wrist_angle, 0, -forearm_offset, forearm_length)

turret_tf = homogeneous(turret_rot, turret_trans)
shoulder_tf = homogeneous(shoulder_rot, shoulder_trans)
arm_tf = homogeneous(arm_rot, arm_trans)
base_to_end_tf = turret_tf * shoulder_tf * arm_tf

display(turret_tf)
display(shoulder_tf)
display(arm_tf)


⎡cos(q₁)  0   -sin(q₁)     0   ⎤
⎢                              ⎥
⎢sin(q₁)  0   cos(q₁)      0   ⎥
⎢                              ⎥
⎢   0     -1     0      d_{o,p}⎥
⎢                              ⎥
⎣   0     0      0         1   ⎦

⎡cos(q₂)  -sin(q₂)  0  d_{q,r}⋅cos(q₂)⎤
⎢                                     ⎥
⎢sin(q₂)  cos(q₂)   0  d_{q,r}⋅sin(q₂)⎥
⎢                                     ⎥
⎢   0        0      1      d_{p,q}    ⎥
⎢                                     ⎥
⎣   0        0      0         1       ⎦

⎡cos(q₃)  -sin(q₃)  0  d_{s,t}⋅cos(q₃)⎤
⎢                                     ⎥
⎢sin(q₃)  cos(q₃)   0  d_{s,t}⋅sin(q₃)⎥
⎢                                     ⎥
⎢   0        0      1     -d_{r,s}    ⎥
⎢                                     ⎥
⎣   0        0      0         1       ⎦

In [182]:
z_0_0 = sp.Matrix([[0], [0], [1]])
z_1_0 = turret_rot * z_0_0
z_2_0 = turret_rot * shoulder_rot * z_0_0
display(z_0_0)
display(z_1_0)
display(z_2_0)

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣1⎦

⎡-sin(q₁)⎤
⎢        ⎥
⎢cos(q₁) ⎥
⎢        ⎥
⎣   0    ⎦

⎡-sin(q₁)⎤
⎢        ⎥
⎢cos(q₁) ⎥
⎢        ⎥
⎣   0    ⎦

In [183]:
z_0_0.cross(translation(base_to_end_tf) - translation(turret_tf))

⎡-d_{p,q}⋅cos(q₁) - d_{q,r}⋅sin(q₁)⋅cos(q₂) + d_{r,s}⋅cos(q₁) + d_{s,t}⋅sin(q₁
⎢                                                                             
⎢-d_{p,q}⋅sin(q₁) + d_{q,r}⋅cos(q₁)⋅cos(q₂) + d_{r,s}⋅sin(q₁) - d_{s,t}⋅sin(q₂
⎢                                                                             
⎣                                                               0             

)⋅sin(q₂)⋅sin(q₃) - d_{s,t}⋅sin(q₁)⋅cos(q₂)⋅cos(q₃)⎤
                                                   ⎥
)⋅sin(q₃)⋅cos(q₁) + d_{s,t}⋅cos(q₁)⋅cos(q₂)⋅cos(q₃)⎥
                                                   ⎥
                                                   ⎦

In [184]:
sp.simplify(jacobian([turret_tf, shoulder_tf, arm_tf], ['revolute', 'revolute', 'revolute']))

⎡-d_{p,q}⋅cos(q₁) - d_{q,r}⋅sin(q₁)⋅cos(q₂) + d_{r,s}⋅cos(q₁) - d_{s,t}⋅sin(q₁
⎢                                                                             
⎢-d_{p,q}⋅sin(q₁) + d_{q,r}⋅cos(q₁)⋅cos(q₂) + d_{r,s}⋅sin(q₁) + d_{s,t}⋅cos(q₁
⎢                                                                             
⎢                                             0                               
⎢                                                                             
⎢                                             0                               
⎢                                                                             
⎢                                             0                               
⎢                                                                             
⎣                                             1                               

)⋅cos(q₂ + q₃)  -(d_{q,r}⋅sin(q₂) + d_{s,t}⋅sin(q₂ + q₃))⋅cos(q₁)  -d_{s,t}⋅si
                                                   